In [46]:
# spotipy modules
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [47]:
# other libraries
from decouple import config 
import time

In [48]:
# visualisation libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
# environment variables
client_id = config('SPOTIPY_CLIENT_ID')
client_secret = config('SPOTIPY_CLIENT_SECRET')
redirect_uri = 'http://google.com/'

In [50]:
# authorization
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [51]:
username = 'sidsaxena'

In [52]:
scope = 'playlist-read-private user-library-read user-top-read user-read-recently-played user-follow-read user-read-currently-playing'

In [65]:
try:
    
    token = util.prompt_for_user_token(username=username, scope=scope, client_id=client_id, client_secret= client_secret, redirect_uri=redirect_uri)
    sp = spotipy.Spotify(auth=token)

except:

    print('Token not accessible for user: ', username)


In [54]:
def getUserPlaylists(user):

    name_list = []
    id_list = []
    creator_list = []

    results = sp.user_playlists(user=username)
    playlists = results['items']
    while results['next']:
        results = sp.next(results)
        playlists.extend(results['items'])
        for playlist, _ in enumerate(playlists):
            name_list.append(playlists[playlist]['name'])
            id_list.append(playlists[playlist]['id'])
            creator_list.append(playlists[playlist]['owner']['id'])
    
    creator_id_tuple = tuple(zip(creator_list, id_list))

    playlist_dict = dict(name = name_list, id = id_list, creator = creator_list)
        
    playlist_df = pd.DataFrame(playlist_dict)
    multiple_playlist_dict = list(creator_id_tuple)
    return playlist_df, multiple_playlist_dict

In [63]:
def analysePlaylist(creator, playlist_id):

    """function to analyse all tracks in a playlist by its ID and create a dataframe with audio features"""
    offset = 0

    while True:
        results = sp.user_playlist_tracks(creator, playlist_id, offset = offset)
        
        playlist = results['items']
        playlist += playlist

        playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]

        playlist_df = pd.DataFrame(columns = playlist_features_list)

    for track in playlist:        
        playlist_features = {}        
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]

        
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index=True)

        if results['next'] is not None:
            offset += 100
        else:
            break 
    return playlist_df

In [56]:
def analysePlaylistsList(playlist_tuple_list):
    """function to analyse multiple playlists"""

    for id, _ in enumerate(playlist_tuple_list):
        playlist_df = analysePlaylist(playlist_tuple_list[id][0], playlist_tuple_list[id][1])

        # playlist_df['playlist'] = PLAYLIST NAME

        if id == 0:
            playlist_tuple_df = playlist_df
        else:
            playlist_tuple_df = pd.concat([playlist_tuple_df, playlist_df], ignore_index=True)

        return playlist_tuple_df

In [57]:
my_playlists_df, my_playlists_list = getUserPlaylists(username)

In [58]:
three_playlists = my_playlists_list[0:3]

In [61]:
three_playlists

[('spotify', '37i9dQZF1E384xMRAb5F4k'),
 ('spotify', '37i9dQZF1E35mQrigIJBb9'),
 ('spotify', '37i9dQZF1E3a2onRCeigm3')]

In [59]:
my_playlists_df.head()

,name,id,creator
0,Daily Mix 1,37i9dQZF1E384xMRAb5F4k,spotify
1,Daily Mix 2,37i9dQZF1E35mQrigIJBb9,spotify
2,Daily Mix 3,37i9dQZF1E3a2onRCeigm3,spotify
3,Daily Mix 4,37i9dQZF1E37kKpoA0xZT0,spotify
4,Daily Mix 5,37i9dQZF1E368h8CDEm5lY,spotify


In [60]:
my_playlists_df.tail()

,name,id,creator
196,Replay Recommendations based on Top Tracks • A...,1Xg90sP3wjKQ6Han8V1t2i,sidsaxena
197,Replay Recommendations based on Top Artists • ...,0gtsnzYKQ1kU93irm6Lepm,sidsaxena
198,90s IndiePop (Hindi),1UsDxFhRXuy3pfRMAoqi1S,21dyhqmkurdn6longfv62uvbi
199,(Alvvays),5NsCCKSRz97bP8OuQskPTg,auuwpwtrrnb9u4xz0sbca61i8
200,Mallsoft & Shopping Mall Ambience,4im2nSUjjCdReLLDNwDQoC,aofd3


In [62]:
multiple_playlists_df = analysePlaylistsList(three_playlists)

UnboundLocalError: local variable 'results' referenced before assignment

In [66]:
test_df = analysePlaylist('sidsaxena', '6Tz22UKQCnp4rqu6dNzVm0')

In [67]:
test_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Benjamin Monday,The Secretary's Daughter,Smoke And Mirrors,2kEd3nFuHAbSLnwfwyi5fW,0.613,0.0197,9,-12.190,0,0.1330,0.968,0.1180,0.4030,71.491,144627,5
1,Steven Wilson,Grace for Drowning,Belle De Jour,3gZLvp9VKl0S96S2Rv81xk,0.362,0.0911,7,-21.017,1,0.0373,0.817,0.1210,0.0353,183.392,179427,4
2,Calum Graham,Tabula Rasa,Farewell (feat. Michael Manring),2wxkIxCjB5047xUG5njMQ2,0.266,0.2330,2,-18.995,1,0.0310,0.957,0.1100,0.0819,88.433,272648,4
3,Steven Wilson,Insurgentes (2016 Remaster),Twilight Coda,5wADBzuyR8NCzmxJWSS7pL,0.375,0.1760,5,-20.460,0,0.0321,0.899,0.0773,0.0320,89.965,204917,4
4,Aeuria,Collection,Norah's Lullaby,07QIkvSRo4ddFlmxniaZsi,0.274,0.1850,11,-19.736,0,0.0362,0.928,0.1050,0.0720,113.893,191634,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Shook,Bicycle Ride,Bicycle Ride,4GkYqdnsWmNPPetlxbhnZ0,0.750,0.3940,9,-12.712,0,0.0348,0.912,0.1860,0.5930,89.965,243120,4
96,The Paper Kites,On the Corner Where You Live,A Gathering on 57th,1cnpE2o3QZSpg6Ma0Sr0cJ,0.520,0.1290,0,-15.577,1,0.0356,0.919,0.3710,0.5100,74.227,90973,3
97,Wave Mage,Breaking the Dream,Breaking the Dream (Lucid Dream Edit),5Z25dBtcA49xWuaXrfaYnH,0.454,0.1780,7,-18.003,1,0.0274,0.324,0.1320,0.1260,90.018,256792,4
98,The Lock-In,The Lock-In,Locked In,30ps6ex21uHnrsnWxXPChA,0.569,0.6110,7,-10.910,0,0.0611,0.881,0.1270,0.6970,193.997,920102,4
